In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import statistics as st
from datetime import datetime
import os

In [ ]:
model_path = "/home/aevans/convLSTM/src/data/temp_df/20240528/Mohawk Valley/20240528_17:35_ml_output.parquet"

clim_div = 'Mohawk Valley'

full_df_path = '/home/aevans/convLSTM/src/data/temp_df/20240331/Mohawk Valley/Mohawk Valley_CAMD.parquet'

In [ ]:
nysm_cats_path = "/home/aevans/nwp_bias/src/landtype/data/nysm.csv"
nysm_cats_df = pd.read_csv(nysm_cats_path)
stations = nysm_cats_df[nysm_cats_df['climate_division_name']==clim_div]['stid'].unique()
stations

In [ ]:
df = pd.read_parquet(model_path)
df = df.sort_index()

In [ ]:
df

In [ ]:
full_df = pd.read_parquet(full_df_path)

In [ ]:
def which_fold(df, fold):
    length = len(df)
    test_len = int(length * 0.2)
    df_train = pd.DataFrame()

    for n in np.arange(0, 5):
        if n != fold:
            df1 = df.iloc[int(0.2 * n * length) : int(0.2 * (n + 1) * length)]
            df_train = pd.concat([df_train, df1])
        else:
            df_test = df.iloc[int(0.2 * n * length) : int(0.2 * (n + 1) * length)]

    return df_train['valid_time'], df_test['valid_time']

In [ ]:
df_train, df_test = which_fold(full_df, 3)

In [ ]:
df_test.iloc[0]

In [ ]:
df_test.iloc[-1]

In [ ]:
vt = full_df['valid_time'].to_list()
vt = vt[:int(len(df)-len(full_df))]
df['valid_time'] = vt

In [ ]:
df = df[df['valid_time'] > datetime(2021, 9, 1, 0, 0, 0)]
df = df[df['valid_time'] < datetime(2021, 9, 30, 23, 0, 0)]
#df = df.drop(columns='valid_time')

In [ ]:
def plot_outputs(df_out, clim_div, stations):
    import matplotlib.pyplot as plt 
    df_out = df_out.sort_index()
    fig, axs = plt.subplots(int(df_out.shape[1]/2), figsize=(21, 21), sharex=True, sharey=True)
    n = int(df_out.shape[1]/2)
    i = 0
    while n>i:
        axs[i].set_ylabel(f'{stations[i]}')
        axs[i].plot(df_out['valid_time'], df_out[f'{i}_target'], c="r", label="Target")
        axs[i].plot(df_out['valid_time'], df_out[f"{i}_convLSTM_output"], c="b", alpha=0.7, label="convLSTM Output")
        
        print(f"Mean Absolute Difference of {stations[i]}: {abs(df_out[f'{i}_target'] - df_out[f'{i}_convLSTM_output']).mean()}")
        i+=1
    fig.suptitle(f"convLSTM Output v Target", fontsize=28)
    axs[0].legend()
    plt.tight_layout()

In [ ]:
plot_outputs(df, clim_div, stations)